# 将常见操作批处理（使用正则来批量选取要操作的文件）

## 设置查找的目录和正则表达式

In [4]:
import os
import tqdm
import shutil

sep = os.sep
# dir_path = r'F:\hcp\HCP'
# dir_path = r'E:\data'
dir_path = r'G:\prep'
dir_path = r'C:\Users\holden\Downloads\dataset-20241019T113733Z-001\dataset\v1\scans'
# dir_patterns= [r'\d+',r'unprocessed',r'3T',r'rfMRI_REST(1|2)_(LR|RL)']
# dir_patterns = [r'\d+']
dir_patterns = [r'(.*)']
# file_pattern = r'^(\d+)_3T_rfMRI_REST(1|2)_(LR|RL)\.nii.gz$'
# file_pattern = r'^(\d+)_3T_rfMRI_REST(1|2)_(LR|RL)\.nii$'
# file_pattern = r'^(\d+_3T_rfMRI_REST(1|2)_(LR|RL)\.nii)$'
file_pattern = r'(.*)\.zip'

## 获取文件路径

In [ ]:
from PathWalker import PathWalker
# pw = PathWalker(dir_path,dir_patterns,file_pattern,True)
# pw.get_path()
# unzip_gzip(pw.res)
pw = PathWalker(dir_path,dir_patterns,file_pattern,True)
pw.get_path()

## 移动文件

In [ ]:
# 输出目录
# out_dir = [r'G:\prep' + os.sep + sublist[1][0] for sublist in pw.collected]
# move_to(pw.res,out_dir)
import shutil
def move_to(matching_files,output_dirs):
    ## move files
    for i,file in tqdm(enumerate(matching_files),total=len(matching_files)):
        if not os.path.isdir(output_dirs[i]):
            os.mkdir(output_dirs[i])
        if not os.path.isfile(output_dirs[i] + os.sep + file.split(os.sep)[-1]):
            shutil.move(file,output_dirs[i])

## 解压文件(zip)

In [5]:
import zipfile

def unzip_file(zip_file_path, extract_to='.'):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

for file in pw.res:
    # the dir of the source file 
    tar_dir = os.sep.join(file.split(os.sep)[0:-2])
    print(tar_dir)
    unzip_file(file,tar_dir)

## 解压文件(gzip)

In [ ]:
import gzip

# 原地解压文件,输出到原来的目录下
for gzip_file in tqdm(pw.res, desc="解压进度", unit="file"):
    output_file = gzip_file[:-3]
    with gzip.open(gzip_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

## 删除文件


In [ ]:
## 设置垃圾箱
recycle_bin = './recycle_bin'
# os.mkdir(recycle_bin)


for file in pw.res:
    # shutil.move(file,recycle_bin)
    print(file)

def delete_file(filename):
    if os.path.exists(filename):
        os.remove(filename)
        print(f"文件 {filename} 删除成功！")
    else:
        print(f"文件 {filename} 不存在。")



In [ ]:
for file in pw.res:
    delete_file(file)

## 保存为mat文件

### 将文件路径（res）和识别到的文件名（filename的第一个捕获组）保存

In [3]:
from P2M import P2M
p2m = P2M()
p2m.save_mat({'filepath':pw.res})